# Capstone Project - BJB - Part 1
### Find the Sweet Spot for your Burger Place in Berlin -- Data Collection and First Exploration

In [19]:
pip install kml2geojson

Note: you may need to restart the kernel to use updated packages.


In [20]:
pip install pykml

Note: you may need to restart the kernel to use updated packages.


In [21]:
pip install shapely

Note: you may need to restart the kernel to use updated packages.


In [22]:
#!conda install -c conda-forge folium=0.5.0 --yes

In [23]:
import folium
import requests # library to handle requests
from pykml import parser
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

### Get public Citizen Data

> We will use zip code wise citizen data.<br>
Therefore we found Berlin zip code polygon data at:  [www.suche-postleitzahl.org](https://www.suche-postleitzahl.org/plz-karte-erstellen) -> geojson file <br>
>We use zip code wise inhabitant numbers also provided by: [www.suche-postleitzahl.org/downloads](https://www.suche-postleitzahl.org/downloads) which is based on German census data from 2011.-> csv file  <br>
>We will load them from our own repository.

In [24]:
df_ct = pd.read_csv('plz_einwohner_berlin.csv') #load citizen data
df_ct = df_ct.rename(columns={'einwohner':'ct_count','plz':'zip'})

berlin_geo = r'plz_5-stellig_berlin.geojson' # load zip code polygons for Berlin

In [25]:
df_ct.head()

,zip,ct_count
0,10115,20313
1,10117,12217
2,10119,16363
3,10178,12167
4,10179,18664


### Get Foursquare Data

In [26]:
CLIENT_ID = 'XXTO3VYG4MWDAGWGWPCXMA2O4HMTTUPOSIEBVU101PHUGVQD' # your Foursquare ID
CLIENT_SECRET = '30K02IQYZP43XYFVE5HW1OP2POGJUQU1L0CHGNV41YS1RBF0' # your Foursquare Secret
VERSION = '20191211'

> In order to obtain a sufficient amount of information on our targets a rectangle around Berlin is defined
> by coordinates and divided in cells. For each cell a request to foursquare is filed. This
> is necessary because there are many items for each of the three requested categories in Berlin
> and foursquare will only provide 50 answers for a single request when using the free developer account.

In [28]:
sw_lon = 13.095 # corners of border around berlin
sw_lat = 52.381
ne_lon = 13.734
ne_lat = 52.677

lons = np.linspace(sw_lon,ne_lon,13) # these are the cell boundaries  for the looping process
lats = np.linspace(sw_lat,ne_lat,13) # we use 12x12 cells, so that there are no more than 50 of ech category in each cell

categorylist = ['4bf58dd8d48988d16c941735','4bf58dd8d48988d1fa931735','4bf58dd8d48988d124941735']#['Burger Joints','Hotels','Office']
categorynames = ['Burger Joints','Hotels','Office']

# we loop over the categories and over the different spatial cells defined above

for idz, category in enumerate(categorylist): # loop over cateories
    for idx in range(len(lons)-1):            # loop over longitude
        for idy in range(len(lats)-1):        # loop over latitude
            url = 'https://api.foursquare.com/v2/venues/search?'

            params = dict(
            client_id=CLIENT_ID,
            client_secret=CLIENT_SECRET,
            v=VERSION,
            intent = 'browse',
            sw = '{}, {}'.format(lats[idy],lons[idx]),
            ne = '{}, {}'.format(lats[idy+1],lons[idx+1]),
            categoryId = category,  #this is Burger Joints
            limit=50
            )

            results = requests.get(url, params=params).json()
            # assign relevant part of JSON to venues
            venues = results['response']['venues']
            if venues != []:

                # tranform venues into a dataframe
                dataframe = json_normalize(venues)
                try:
                    new_df=dataframe[['name','location.lat','location.lng','location.postalCode','id']].copy()
                    new_df.rename({'location.lat':'lat','location.lng':'lon','location.postalCode':'zip'},axis=1,inplace=True)
                except:
                    pass
                if 'res_df' not in locals():
                    res_df = new_df.copy()
                else:
                    res_df = res_df.append(new_df)
            else:
                pass
    
    res_df.drop_duplicates(inplace= True)
    res_df.reset_index(drop = True,inplace= True)  
    #write into different dataframes 
    if idz == 0:
        df_bj = res_df.copy()#burgerjoints
        del res_df
    elif idz == 1:
        df_ht = res_df.copy()#hotels
        del res_df
    elif idz == 2:
        df_of = res_df.copy()#offices
        del res_df  
    

In [29]:
df_bj.head(5)

,name,lat,lon,zip,id
0,Kladow Grill Burger Pizza,52.453026,13.141852,14089,4f92ddb6e4b008256552e140
1,Volcano Burger,52.525032,13.196861,13595,594d55f69d6a19266e4509f0
2,Cruise-In,52.532584,13.178774,13581,4bf2815baf659c744ffcd747
3,"Bastis Currys, Burgers & Fries",52.533400,13.190330,13581,5a17e311bfc6d03f268daa13
4,Burger Route,52.530632,13.196140,13581,5921ba9c89e49063753304e5


In [30]:
df_ht.head(5)

,name,lat,lon,zip,id
0,Breakfast Room,52.399327,13.112184,NaN,5dd0f776ed8bfb0008fd2cf8
1,Concorde Hotel Forsthaus,52.405273,13.140936,14109,4ea0073e8b816206b9f66786
2,Pension Zeitlos,52.392231,13.098609,14482,4bc793c193bdeee1538837ae
3,avendi Hotel am Griebnitzsee,52.395366,13.127460,14482,4b0b2b48f964a520fc2d23e3
4,Apartements Filmeck,52.382681,13.122606,14480,57a37946498e72a672128ef4


In [31]:
df_of.head(5)

,name,lat,lon,zip,id
0,MedienHaus Babelsberg - Zentrum für Film- und ...,52.387894,13.119310,14482,506daff2e4b0dd5636405aae
1,VCAT Consulting HQ,52.388084,13.119418,14482,4bdae2aac79cc9286c1880e9
2,Vragments,52.388901,13.120517,14482,5b71911df2554e002c9e3f9b
3,Point Cloud Technology GmbH,52.387762,13.121250,14482,5cc6ba94c0af57002cd0a6ac
4,Radio TEDDY HQ,52.382264,13.120453,14482,4b90c0a2f964a520419633e3


> We note, that there are missing zip code values in the data frames. We have to replace them.
> Simultaneously we drop all places that are not in Berlin, as we compare with Berlin zip code polygons. 

In [32]:
def addmissingzips(df):
    '''function to add missing zip values'''
    zip_df = pd.DataFrame([],columns= ['name','lat','lon','zip','id'])
    with open('plz_5-stellig_berlin.kml') as f:
      folder = parser.parse(f).getroot().Document.Folder
    i = 0
    for pm in folder.Placemark:
        try:
            pmstr = pm.Polygon.outerBoundaryIs.LinearRing.coordinates
            slist = [s.split(',') for s in str(pmstr).split(' ')]
            sarray = np.array(slist).astype('float')
        except AttributeError:
            pmstr = pm.MultiGeometry.Polygon.outerBoundaryIs.LinearRing.coordinates
            slist = [s.split(',') for s in str(pmstr).split(' ')]
            sarray = np.array(slist).astype('float')

        for lat, lon, name,ID in df[['lat','lon','name','id']].values:
            point = Point(lon,lat)
            polygon = Polygon(sarray)
            flag = polygon.contains(point)

            if flag:
                zip_df.loc[i] =[name,lat,lon,int(pm.name),ID]            
                i+=1  
    return zip_df


In [33]:
# here we add the missing zips for each category
df_bj = addmissingzips(df_bj)
df_ht = addmissingzips(df_ht)
df_of = addmissingzips(df_of)

In [34]:
# and we store the data frames as csv in our local repository
df_bj.to_csv('cpstn_bj.csv',index=False)
df_ht.to_csv('cpstn_ht.csv',index=False)
df_of.to_csv('cpstn_of.csv',index=False)

> Let us get one first glance at the entire data base.

In [35]:
df_bj.describe()

,name,lat,lon,zip,id
0,Tommi's Burger Joint,52.532514,13.397578,10115,528a4e3411d24e4d2d087a64
1,Hasir Burger,52.532432,13.380729,10115,5704e0dc498ec9949606a33c
2,Peter Pane,52.521195,13.388127,10117,56e6f680498e5021b8c11ee8
3,Vedang,52.510559,13.380508,10117,5ccc141b3fffb400256e2f79
4,BraufactuM Berlin am Hausvogteiplatz (ehemals ...,52.513420,13.396050,10117,5c9d80a13183940025c9a9e8


In [36]:
df_ht.describe()

,name,lat,lon,zip,id
0,Titanic Chaussee Berlin,52.532215,13.381012,10115,56a12a9b498eedc0ca6aa26a
1,INNSiDE Berlin Mitte,52.532891,13.380609,10115,5b5744cec824ae002ca934c6
2,Mercure Hotel Berlin City,52.530475,13.381626,10115,4b7d86c4f964a5207fc42fe3
3,H+ Hotel Berlin Mitte,52.529586,13.383976,10115,4b1d6047f964a5209e0f24e3
4,Hotel i31,52.531107,13.384270,10115,510fd821d86c62b891ed167b


In [37]:
df_of.describe()

,name,lat,lon,zip,id
0,Packagist Conductors,52.525776,13.399334,10115,5b965779a4ba7c002ceeb06f
1,Factory Berlin,52.537174,13.394512,10115,4fcd0618e4b09fb19cb92b50
2,Silicon Allee Campus,52.530236,13.383890,10115,4fa7a540e4b0752c6af21d24
3,HERE Campus,52.530843,13.384931,10115,4f16a28fe4b0044a285b0d53
4,SoundCloud HQ,52.537310,13.394305,10115,536539ed498e3a9213a33acb


> This concludes the data acquisition.